# 1. Overview

Below are the variables that the data set contains:
* **vendor_id**:	A code indicating the TPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc
* **pickup_datetime**: The date and time when the meter was engaged.
* **dropoff_datetime**: The date and time when the meter was disengaged
* **passenger_count**: The number of passengers in the vehicle. This is a driver-entered value
* **trip_distance**: The elapsed trip distance in miles reported by the taximeter.
* **rate_code**: The final rate code in effect at the end of the trip. 1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride
* **store_and_fwd_flag**: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip N= not a store and forward trip
* **payment_type**:	A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip
* **fare_amount**: The time-and-distance fare calculated by the meter
* extra	Miscellaneous extras and surcharges. Currently, this only includes the \$0.50 and \$1 rush hour and overnight charges.
* **mta_tax**: \$0.50 MTA tax that is automatically triggered based on the metered rate in use
* **tip_amount**:	Tip amount – This field is automatically populated for credit card tips. Cash tips are not included
* **tolls_amount**:	Total amount of all tolls paid in trip.
* **imp_surcharge**:	\$0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
* **total_amount**:	The total amount charged to passengers. Does not include cash tips
* **pickup_location_id**: TLC Taxi Zone in which the taximeter was engaged
* **dropoff_location_id**: TLC Taxi Zone in which the taximeter was disengaged

# 2. Getting Started

In [1]:
# Imports 
import numpy as np
import pandas as pd

from pyspark.sql import SQLContext
from pyspark.sql import types
from pyspark.sql.functions import isnan, when, count, col

In [2]:
# Initialize Spark
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [116]:
# Get data from BigQuery and store it into Cloud Storage as GZIP compressed .csv files
!bq --location=US extract --compression GZIP 'bigquery-public-data:new_york_taxi_trips.tlc_yellow_trips_2018' gs://ajk-portfolio/data/nyc_taxi_trips/nyc_taxi_2018-*.csv.gz
!bq --location=US extract --compression GZIP 'bigquery-public-data:new_york_taxi_trips.taxi_zone_geom' gs://ajk-portfolio/data/nyc_taxi_trips/nyc_taxi_zones-*.csv.gz
!bq --location=US extract --compression GZIP 'bigquery-public-data:noaa_gsod.stations' gs://ajk-portfolio/data/nyc_weather/noaa_stations-*.csv.gz
!bq --location=US extract --compression GZIP 'bigquery-public-data:noaa_gsod.gsod2018' gs://ajk-portfolio/data/nyc_weather/noaa_weather-*.csv.gz

Waiting on bqjob_r5ae5d0098852883d_00000169b23015d0_1 ... (85s) Current status: DONE   


In [3]:
# Load data into Spark DataFrame
trips_df = spark.read \
                .option("header", "true") \
                .option("inferSchema", "true") \
                .csv("gs://ajk-portfolio/data/nyc_taxi_trips/nyc_taxi_2018-*.csv.gz")

loc_df = spark.read \
              .option("header", "true") \
              .option("inferSchema", "true") \
              .csv("gs://ajk-portfolio/data/nyc_taxi_trips/nyc_taxi_zones-*.csv.gz")

stations_df = spark.read \
                   .option("header", "true") \
                   .option("inferSchema", "true") \
                   .csv("gs://ajk-portfolio/data/nyc_weather/noaa_stations-*.csv.gz")

weather_df = spark.read \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .csv("gs://ajk-portfolio/data/nyc_weather/noaa_weather-*.csv.gz")

In [4]:
# Print schemas of each DataFrame to ensure the data got loaded correctly correctly
trips_df.printSchema()
loc_df.printSchema()
stations_df.printSchema()
weather_df.printSchema()

root
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- imp_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)

root
 |-- zone_id: integer (nullable = true)
 |-- zone_name: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- zone_geom: string (nullable = true)

root
 |-- usaf: string (nullable = true)
 |-- wban: integ

In [5]:
# register temp table so it can be queried with SQL
trips_df.registerTempTable("trips")
loc_df.registerTempTable("taxi_locations")
stations_df.registerTempTable("weather_stations")
weather_df.registerTempTable("weather")

In [22]:
# Join in all the loaded DataFrames into one
query = """
WITH daily_weather AS
    (SELECT
        TO_DATE(CAST(UNIX_TIMESTAMP(CONCAT(w.mo,'/',w.da,'/',w.year), 'MM/dd/yyyy') AS TIMESTAMP)) AS weather_date,
        w.temp AS avg_temp,
        w.max AS max_temp,
        w.min AS min_temp,
        w.prcp AS percip_inch,
        IF(w.sndp = 999.9, 0, w.sndp) AS snow_depth,
        w.fog,
        w.rain_drizzle,
        w.snow_ice_pellets,
        w.hail,
        w.thunder,
        w.tornado_funnel_cloud,
        w.wdsp AS wind_speed,
        w.dewp,
        w.visib
    FROM
        weather w
    WHERE 
        w.wban IN 
            (SELECT
                ws.wban
            FROM 
                weather_stations ws
            WHERE
                ws.country = 'US'
                AND ws.state = 'NY'
                AND ws.call = 'KNYC')),
yellow_trips AS 
    (SELECT
        t.vendor_id,
        TO_DATE(t.pickup_datetime) AS pickup_date,
        t.pickup_datetime,
        TO_DATE(t.dropoff_datetime) AS dropoff_date,
        dropoff_datetime,
        t.passenger_count,
        t.trip_distance,
        t.rate_code,
        t.store_and_fwd_flag, 
        t.payment_type,
        t.fare_amount,
        t.extra, 
        t.mta_tax, 
        t.tip_amount, 
        t.tolls_amount, 
        t.imp_surcharge,
        t.total_amount, 
        t.pickup_location_id, 
        ptl.borough AS pickup_borough,
        ptl.zone_name AS pickup_zone,
        t.dropoff_location_id,
        dtl.borough AS dropoff_borough,
        dtl.zone_name AS dropoff_zone
    FROM
        trips t
    LEFT JOIN
        taxi_locations ptl
        ON 
            ptl.zone_id = t.pickup_location_id
    LEFT JOIN
        taxi_locations dtl
        ON
            dtl.zone_id = t.dropoff_location_id)
SELECT 
    yt.vendor_id,
    yt.pickup_date,
    yt.pickup_datetime,
    yt.dropoff_date,
    yt.dropoff_datetime,
    yt.passenger_count,
    yt.trip_distance,
    yt.rate_code,
    yt.store_and_fwd_flag,
    yt.payment_type,
    yt.fare_amount,
    yt.extra,
    yt.mta_tax,
    yt.tip_amount,
    yt.tolls_amount,
    yt.imp_surcharge,
    yt.total_amount,
    yt.pickup_location_id,
    yt.pickup_borough,
    yt.pickup_zone,
    yt.dropoff_location_id,
    yt.dropoff_borough,
    yt.dropoff_zone,
    dw.avg_temp,
    dw.max_temp,
    dw.min_temp,
    dw.percip_inch,
    dw.snow_depth,
    dw.fog,
    dw.rain_drizzle,
    dw.snow_ice_pellets,
    dw.hail,
    dw.thunder,
    dw.tornado_funnel_cloud,
    dw.wind_speed,
    dw.dewp,
    dw.visib
FROM 
    yellow_trips yt
LEFT JOIN
    daily_weather dw
    ON 
        dw.weather_date = yt.pickup_date
"""

# create a new DataFrame based on the query
combo_df = spark.sql(query)

# Register the new DataFrame so that we can do EDA
combo_df.registerTempTable("combo")

# 3. Data Quality Check

For the sake of brevity, I am not including the various steps I took for data quality check that was done prior to doing data analysis. However, the following are the summary of some of the important findings:
* There are a handful of records where the `total_amount` paid for the is less than 0 or exceeds \$100,000. This does not make a lot of sense given the data set, and seems to likely be caused by erroneous data entry. 
* There are few records where the `trip_distance` is either 0 or greater than 100,000 miles. the 0 miles travelled may be due to cancelled trips, while the ones where over 100,000 miles is traveled are likely caused by erroneous entries.
* There are records where the `passenger_count` have odd values: 0 or greater than 100. Since this value is entered by taxi drivers, the 0s may be instances where the driver did not enter any value, and the ones over 100 may be erroneous entries.
* The data set also seems to be incomplete, as there are data from odd years (e.g. 1989) and the trips are mainly from January-June 2018. There also seems to be only less than 200 trips for July 2018 - December 2018.

The problematic records above are all removed prior to any subsequent analysis done as part of this notebook. In total 43K records were removed out of 63 million+ that the dataset contains.


In [24]:
# Filtering out problematic data
query = """
SELECT
    *
FROM
    combo c
WHERE 
    c.total_amount < 10000
    AND c.total_amount > 0
    AND c.trip_distance < 1000
    AND c.passenger_count < 100
    AND c.pickup_date >= '2018-01-01'
    AND c.pickup_date < '2018-07-01'
"""
# create a new DataFrame based on the query
combo_df = spark.sql(query)

# Re-register the new DataFrame so that we can do EDA based on the clean data
combo_df.registerTempTable("combo")

# 4. Exploratory Data Analysis

* There seems to be records where the total amount exceeds $100,000, which does not make a lot of sense. Likely these records will have to be removed prior to any modelling work.
* There seems to be a few issues with the `trip_distance` variables as there are instances where the logged value is 0 or greater than 100,000 miles. These are likely to be erroneous values and will be removed from the DataFrame
* There seems to be

### How many passengers do trips typically have?

In [25]:
# get summary statistics for passenger_count
combo_df.describe(['passenger_count']).show()

+-------+------------------+
|summary|   passenger_count|
+-------+------------------+
|  count|          63330490|
|   mean| 1.599967535384615|
| stddev|1.2476888566157074|
|    min|                 0|
|    max|                 9|
+-------+------------------+



### How far are the distance for the taxi trips?
It looks like on average each trip is about 3 miles.

In [26]:
# get summary statistics for passenger_count
combo_df.describe(['trip_distance']).show()

+-------+------------------+
|summary|     trip_distance|
+-------+------------------+
|  count|          63330490|
|   mean| 2.895500479784452|
| stddev|3.7459989721729565|
|    min|               0.0|
|    max|             943.5|
+-------+------------------+



### What payment methods are popular?

In [27]:
query = """
SELECT DISTINCT
    CASE WHEN c.payment_type = 1 THEN "Credit Card"
         WHEN c.payment_type = 2 THEN "Cash"
         WHEN c.payment_type = 3 THEN "No Charge"
         WHEN c.payment_type = 4 THEN "Dispute"
         WHEN c.payment_type = 5 THEN "Unknown"
         WHEN c.payment_type = 6 THEN "Voided Trip" 
         END AS payment_type,
    COUNT(*) OVER(PARTITION BY c.payment_type) AS trip_count,
    ROUND((COUNT(*) OVER(PARTITION BY c.payment_type)/COUNT(*) OVER())*100, 2) AS pct_total,
    ROUND(AVG(c.fare_amount) OVER(PARTITION BY c.payment_type), 2) AS avg_fare_amount,
    ROUND(AVG(c.trip_distance) OVER(PARTITION BY c.payment_type), 2) AS avg_trip_distance,
    ROUND(AVG(c.passenger_count) OVER(PARTITION BY c.payment_type), 2) AS avg_passenger_count
FROM
    combo c
ORDER BY
    trip_count DESC
"""

spark.sql(query).toPandas()

,payment_type,trip_count,pct_total,avg_fare_amount,avg_trip_distance,avg_passenger_count
0,Credit Card,44131674,69.68,13.22,3.03,1.59
1,Cash,18806396,29.70,11.70,2.57,1.63
2,No Charge,311721,0.49,13.36,2.93,1.20
3,Dispute,80699,0.13,14.16,3.19,1.20


### What are the top trip destinations? and where are those trips coming from?
Below is a results table that contain the top 3 destinations along with the top 5 origin for each of the 3 destinations. A few observation on the results:
* It looks like in NYC, taxis are mainly used for short/medium distance transportation since the majority of the trips are fairly short in distance (around 1 mile or less), and the fare amount is relatively low (around \$10)
* In addition majority of the destination and origin are within Manhattan, which is the most populous borough in NYC.


In [28]:
query = """
WITH top_destination_origin AS 
    (SELECT
        c.dropoff_borough,
        c.dropoff_zone,
        c.pickup_borough,
        c.pickup_zone,
        COUNT(*) trip_count,
        AVG(c.trip_distance) AS avg_trip_distance,
        AVG(c.passenger_count) AS avg_passenger_count,
        AVG(c.fare_amount) AS avg_fare_amount,
        DENSE_RANK() OVER(PARTITION BY c.dropoff_borough, c.dropoff_zone ORDER BY COUNT(*) DESC) AS trip_count_rank
    FROM
        combo c
    GROUP BY
        c.pickup_borough,
        c.pickup_zone,
        c.dropoff_borough,
        c.dropoff_zone),
top_destination AS 
    (SELECT
        c.dropoff_borough,
        c.dropoff_zone,
        DENSE_RANK() OVER(ORDER BY COUNT(*) DESC) AS trip_count_rank,
        COUNT(*) AS trip_count
    FROM 
        combo c
    GROUP BY
        c.dropoff_borough,
        c.dropoff_zone)
SELECT
    tdo.dropoff_borough,
    tdo.dropoff_zone,
    tdo.pickup_borough,
    tdo.pickup_zone,
    tdo.trip_count,
    tdo.avg_trip_distance,
    tdo.avg_passenger_count,
    tdo.avg_fare_amount
FROM 
    top_destination td
INNER JOIN 
    top_destination_origin tdo
    ON 
        tdo.dropoff_borough = td.dropoff_borough
        AND tdo.dropoff_zone = td.dropoff_zone
WHERE
    td.trip_count_rank IN (1, 2, 3) 
    AND tdo.trip_count_rank IN (1, 2, 3, 4, 5)
ORDER BY
    td.trip_count_rank,
    tdo.trip_count_rank
"""

spark.sql(query).toPandas()

,dropoff_borough,dropoff_zone,pickup_borough,pickup_zone,trip_count,avg_trip_distance,avg_passenger_count,avg_fare_amount
0,Manhattan,Upper East Side North,Manhattan,Upper East Side South,371383,1.040262,1.572988,6.284421
1,Manhattan,Upper East Side North,Manhattan,Upper East Side North,322439,0.614163,1.572161,5.141416
2,Manhattan,Upper East Side North,Manhattan,Lenox Hill West,147456,1.081683,1.608921,6.341173
3,Manhattan,Upper East Side North,Manhattan,Yorkville West,134962,0.673250,1.601732,5.222884
4,Manhattan,Upper East Side North,Manhattan,Midtown Center,112407,1.991236,1.580329,10.214633
5,Manhattan,Midtown Center,Manhattan,Upper East Side South,155187,1.087860,1.545664,7.888813
6,Manhattan,Midtown Center,Manhattan,Penn Station/Madison Sq West,138413,1.307035,1.568465,9.091393
7,Manhattan,Midtown Center,Manhattan,Midtown Center,125097,0.619652,1.563123,6.858843
8,Manhattan,Midtown Center,Manhattan,Garment District,106948,0.964250,1.549417,7.055784
9,Manhattan,Midtown Center,Manhattan,Murray Hill,104056,0.933680,1.555768,7.388775


### How does taxi usage vary by Month?
There is some oddity on the results here, it looks like the number of trips in March 2018 is double the amount than any of the other month: there are ~19 million trips in March 2018 alone, as opposed to it typically being in the 8-9 million range. I currently do not have a good explanation why there is such a stark difference in the trip count in March 2018, but this is something that needs to be explored further.


In [30]:
query = """
SELECT
    DATE_TRUNC('MM', c.pickup_date) AS date_month,
    COUNT(*) AS trip_count
FROM
    combo c
GROUP BY
    date_month
ORDER BY
    date_month
"""

spark.sql(query).toPandas()

,date_month,trip_count
0,2018-01-01,8757334
1,2018-02-01,8490206
2,2018-03-01,18850979
3,2018-04-01,9302068
4,2018-05-01,9220545
5,2018-06-01,8709358


### How does the weather impact ridership?

In [ ]:
#TODO

# 5. Models

In [ ]:
# predicting travel times
# predicting total tip
# pickup dropoff location web graph

# 6. Conclusion